In [2]:
################################
#                              #
#          INPUT               #
#                              #
################################
alpha_IMF = 1.35 # Salpeter IMF
z_max = input('Give a value of the maximum redshift used :')
fow_LSST = 3.5 #field of view LSST in degrees (*60 in arcmin)
delta_omega = fow_LSST*1.0/pow((180/math.pi),2) #field of view DECam in str
mag_lim = input('Give a value for limit magnitude ')# apparente magnitde limite for the LSST   
d_h = c_light*1.0/70 #Hubble distance in Mpc

def e_z_distri(z):
    return (1.0/math.sqrt(omega_m*((1+z)**3)+ omega_k*((1+z)**2) + omega_lambda))
    e_z_int, e_z_int_err = integrate.quad(e_z_distri,0.,z)

def r_com_distri(z):                   # We define comoving distance for flat universe in Mpc
    return (d_h*integrate.quad(e_z_distri,0.,z)[0])

def d_lum_distri(z):                   # We define luminosity distance for flat universe in Mpc
    return (r_com_distri(z)*(1+z))

def d_mod_distri(z):                   # We define modulus distance for flat universe
    return (5.0 * np.log10(d_lum_distri(z)*10**6/10))

def madau_time(t):                   # We define t9 as Madeau et al 1998
    return (13*pow((1+t),-1.5))

########################################################################################################################

                 ########################################
                 #                                      #
                 #           IMF(Salpeter)              #
                 #                                      #
                 ########################################

           #REFERENCE : Salpeter et al. 1955 ApJ 121: 161

########################################################################################################################

def salpeter(M):
    return (pow(M,-(1+alpha_IMF)))
def Mass_tot(M):
    return (pow(M,-alpha_IMF))

mass_min=8
mass_max=50

#X_SN/<mSN> dans l article de Lien 2009
'''

CCSNe rate est directement proportionelle a la formation stellaire
CCSNe_rate=IMF*SFR

'''
IMF=integrate.quad(salpeter,mass_min,mass_max)[0]/(integrate.quad(Mass_tot,0.1,100))[0] #number of stars with M>8Msol

########################################################################################################################

                     ########################################
                     #                                      #
                     #           Star Formation Rate        #
                     #                   &                  #
                     #             Supernove Rate           #
                     ########################################

         #REFERENCE : 
         #            Horiuchi et al. 2011 ApJ, 738, 154H
         #            Cole et al. 2001  MNRAS, 326, 255C
         
########################################################################################################################
a=3.4
b=-0.3
c=-3.5
ro=0.016*0.73
eta=-10
B=2**(1-a/b)
C=2**((b-a)/c)*5**(1-b/c)

def SFR_Horiuchi(z):
    return (ro*((1+z)**(a*eta)+((1+z)*1.0/B)**(b*eta)+((1+z)*1.0/C)**(c*eta))**(-0.1))

a1=0.017
b1=0.13
c1=3.3
d1=5.3


def SFR_Cole(z):
    return (h*(a1+b1*z)/(1+(z/c1)**d1))

SFR_Horiuchi_vect=vectorize(SFR_Horiuchi)
SFR_Cole_vect=vectorize(SFR_Cole)

z_sfr=np.arange(0,5,0.001)
plot(z_sfr,np.log10(SFR_Horiuchi_vect(z_sfr)),'g',z_sfr,np.log10(SFR_Cole_vect(z_sfr)),'k')
legend(('Horiuchi','Cole'),2)
xlabel('redshift')
ylabel('log SFR')
show()

########################################################################################################################
                     ########################################
                     #                                      #
                     #              Filters                 #
                     #              LSST=SDSS               #
                     ########################################
########################################################################################################################

                  
trans=np.loadtxt('DECam_transmission2.dat')
lambda_filter=trans[:,0]*10 #in AA
s_u=trans[:,1]/100 # not in %
s_g=trans[:,2]/100 # not in %
s_r=trans[:,3]/100 # not in %
s_i=trans[:,4]/100 # not in %
s_z=trans[:,5]/100 # not in %

u_func=interpolate.interp1d(lambda_filter,s_u)
g_func=interpolate.interp1d(lambda_filter,s_g)
r_func=interpolate.interp1d(lambda_filter,s_r)
i_func=interpolate.interp1d(lambda_filter,s_i)
z_func=interpolate.interp1d(lambda_filter,s_z)

dem_u=integrate.simps(u_func(lambda_filter)*1.0/lambda_filter,lambda_filter) 
dem_g=integrate.simps(g_func(lambda_filter)*1.0/lambda_filter,lambda_filter) 
dem_r=integrate.simps(r_func(lambda_filter)*1.0/lambda_filter,lambda_filter) 
dem_i=integrate.simps(i_func(lambda_filter)*1.0/lambda_filter,lambda_filter) 
dem_z=integrate.simps(z_func(lambda_filter)*1.0/lambda_filter,lambda_filter) 

plot(lambda_filter,s_u,'c',lambda_filter,s_g,'b',lambda_filter,s_r,'r',lambda_filter,s_i,'m',lambda_filter,s_z,'k')
legend(('DES-u','DES-g','DES-r','DES-i','DES-z'),loc=1,markerscale=0.5,prop={'size':10},ncol=3) 
xlabel('wavelength [\AA]')
ylabel('Transmission')
show()

########################################################################################################################

                     ########################################
                     #                                      #
                     #             K-correction             #
                     #                                      #
                     ########################################

         #REFERENCE : 
	 #            Lien and Field 2009
         #            Dessart et al. 2013 
         #            de Jaeger et al. 2015-16           
########################################################################################################################

'''

On prend le modele de Luc pour faire la K-correction, on prend le modele 50 jours apres explosion

'''
#model de Luc Dessart
Model=[str('m15_du_sch_mlt3_FeC_mix0p4_18_LR_1500_50000.fl')]
epoque_model=51
txt=np.loadtxt('/home/thomas/Recherche/THESE_LC/K_correction/Models_IF/Luc/%s'%Model[0]).transpose()
#rest frame
lambda_model=(txt[0])
flux_model=(txt[1])
F_spec_model_0=interpolate.interp1d(lambda_model,flux_model)
#Filter r
lambda_r=lambda_filter[150:1000]
#grille redshift
z_sim=np.arange(0.01,z_max,0.01)
K_r=np.zeros(size(z_sim))
for  i in range(size(z_sim)):
	lambda_model_obs=lambda_model*(1+z_sim[i])
	flux_model_obs=flux_model*1.0/(1+z_sim[i])
	F_spec_model_z=interpolate.interp1d(lambda_model_obs,flux_model_obs)
	#On va comparer F_spec_model_z(des) et F_spec_model_0(des)
	n_des=integrate.simps(F_spec_model_z(lambda_r)*r_func(lambda_r)*lambda_r,lambda_r)   
	m_des_z=-2.5*(np.log10(n_des/dem_r))-48.60+2.5*(np.log10(c_AA))

	#On va comparer F_spec_model_z(SDSS) et F_spec_model_0(CSP)
	n_des=integrate.simps(F_spec_model_0(lambda_r)*r_func(lambda_r)*lambda_r,lambda_r)   
	m_des_0=-2.5*(np.log10(n_des/dem_r))-48.60+2.5*(np.log10(c_AA))

	K_r[i]=m_des_z-m_des_0

Kr_func=interpolate.interp1d(z_sim,K_r)

#plot(z_sim,K_r,'ro')
#xlabel('Redshift')
#ylabel('Kr')
#show()

########################################################################################################################

                     ########################################
                     #                                      #
                     #           Effect of Dust             #
                     #                                      #
                     ########################################

         #REFERENCE : 
         #            Lien and Field 2009
########################################################################################################################

def dust(z):
	if z<3.3:
		f_dust=0.95-0.28*z
	else:
		f_dust=0.02
	return f_dust 




#######  INPUTS #######

#Grid of redshift

####################################################################################################################
#
#	On telecharge le model de Luc Dessart
#
###############################################################################

####### Lambda effective
	## Liste des differents epoques modele  ###
	
Model=[str('m15_du_sch_mlt3_FeC_mix0p4_3_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_4_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_5_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_6_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_7_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_8_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_9_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_10_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_11_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_12_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_13_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_14_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_15_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_16_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_17_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_18_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_19_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_20_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_21_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_22_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_23_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_24_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_25_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_26_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_27_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_28_LR_1500_50000.fl'),str('m15_du_sch_mlt3_FeC_mix0p4_29_LR_1500_50000.fl')]
epoque_model=[12.2,13.41,14.75,16.23,17.86,19.65,21.62,23.78,26.16,28.78,31.66,34.83,38.31,42.14,46.35,51,56.1,61.71,67.88,74.67,82.14,90.35,99.4,109.4,120.3,129,133]


lam=size(epoque_model)*[0]
flux_model=size(epoque_model)*[0]

for  i in range(size(Model)):
	txt=np.loadtxt('/home/thomas/Recherche/THESE_LC/K_correction/Models_IF/Luc/%s'%Model[i]).transpose()
	lam[i]=(txt[0])
	flux_model[i]=(txt[1])*1*10**8

########################################
#
# Apparent magnitude m(time,z,E(B-V))
#
#m=-2.5log [ integral(S_lambda*F_lambda*lambda*d_lambda)/integral(S_lambda/lambda*d_lambda)]-48.06+2.5log(3.0e18)
#S_lambda=filter transmission function
#F_lambda=flux in lambda
#At the end 3.0e18= light velocity in AA/s
#
#F_lambda=L_lambda_rest_frame/(4*pi*dL**2)*1/(1+z)
#

dem_r=integrate.simps(r_func(lambda_filter)*1.0/lambda_filter,lambda_filter) 

num=np.zeros(shape=(size(epoque_model),size(z_sim)))
m_sn_model=np.zeros(shape=(size(epoque_model),size(z_sim)))
for i in range(size(epoque_model)):
    for t in range(size(z_sim)):
        dem=4*math.pi*(d_lum_distri(z_sim[t])*1.0e6)**2 #4pi*dL(z) where dL is in cm
	lambda_model_obs=lam[i]*(1+z_sim[t])
	flux_model_obs=flux_model[i]*1.0/(1+z_sim[t])
	F_spec_model_z=interpolate.interp1d(lambda_model_obs,flux_model_obs)
        num[i,t]=integrate.simps(F_spec_model_z(lambda_r)*r_func(lambda_r)*lambda_r,lambda_r)*1.0/dem
        m_sn_model[i,t]=-2.5*np.log10((num[i,t]/dem_r))-48.60+2.5*(np.log10(c_AA))-1.086*log(dust(z_sim[t]))


####
#
# On charge la courbe de lumiere de SN 2005J
#
#######
JD_explosion=2453382.78
JD_ref=2453000
z_cmb=4183*1.0/c_light
err_z_cmb=150*1.0/c_light
SN_photo=np.loadtxt('../CSP/Raw_Data_CSP/SN05J.dat').transpose()
SN_epoch=SN_photo[0][SN_photo[11]!=999.0]+JD_ref-JD_explosion
mag_V=SN_photo[11][SN_photo[11]!=999.0]
dmag_V=SN_photo[12][SN_photo[11]!=999.0]

M_SN_r=mag_r-5*log10(d_lum_distri(z_cmb))-25
dM_SN_r=sqrt(dmag_r**2+(np.array(err_z_cmb)*(5*(1+np.array(z_cmb))*1.0/(np.array(z_cmb)*(1+np.array(z_cmb)*1.0/2)*log(10))))**2)


m_sn=np.zeros(shape=(size(SN_epoch),size(z_sim)))
for i in range(size(SN_epoch)):
    for t in range(size(z_sim)):
        m_sn[i,t]=M_SN_r[i]+d_mod_distri(z_sim[t])

plot(SN_epoch,-m_sn[:,80])

#############
###
### We now compare our m(time,z) with m_lim of the camera
###-----> We obtain delta_t(z)
###
##############

t_min=[0]*size(z_sim)
t_max=[0]*size(z_sim)
t_max_2=[0]*size(z_sim)
delta_t_redshift=[0]*size(z_sim) #delta t corrected by redshift
epoque_model_z=[0]*size(SN_epoch)


for i in range(size(z_sim)):
    epoque_model_z=SN_epoch*(1+z_sim[i])  #1+z for the time
    if min(m_sn[:,i][:])<mag_lim:
        t_min[i]=epoque_model_z[min(np.where(m_sn[:,i]<=mag_lim)[0])]
        o=set(np.where(epoque_model_z>=t_min[i])[0])#index where last t>t_min
        p=set(np.where(m_sn[:,i]<=mag_lim)[0])#index where we see the SN
        q=(list(o-p))#all the index where we see the pic of the shock breakout
        if q==[]:
            t_max[i]=epoque_model_z[max(np.where(m_sn[:,i]<=mag_lim)[0])]
            delta_t_redshift[i]=t_max[i]-t_min[i]
        else:
            t_max[i]=epoque_model_z[min(q)]
            t_max_2[i]=epoque_model_z[max(q)]
            delta_t_redshift[i]=(t_max[i]-t_min[i])+(max(epoque_model_z)-t_max_2[i])#le SBO+ rest of curve


##def SNR(z):
##    return (IMF*SFR_model(z))

def SNR_Horiuchi(z):
    return (IMF*SFR_Horiuchi(z))# Horiuchi
def SNR_Cole(z):
    return (IMF*SFR_Cole(z)) # Cole

########################################################################################################################

########################################
#              dN/dz                   #
########################################

########################################################################################################################


dV_dz_dW=np.zeros(size(z_sim))
dn_dz_Horiuchi=np.zeros(size(z_sim))  # Madau model 1
n_Horiuchi=[0]*size(z_sim)
dn_dz_Cole=np.zeros(size(z_sim)) # Madau model 2
n_Cole=[0]*size(z_sim)
# The number of detected SNe II per unit of redshift interval is    

for i in range(size(z_sim)):
    dV_dz_dW[i]=(pow(r_com_distri(z_sim[i]),2)*d_h*e_z_distri(z_sim[i]))    #dV(z) is the volume per unit redshift being surveyed
    dn_dz_Horiuchi[i]=(SNR_Horiuchi(z_sim[i])*1.0/(1+z_sim[i])*dV_dz_dW[i]*delta_omega)#dn/dz is the SN per unit of redshift, comovil volume, solid angle and fied of view
    dn_dz_Cole[i]=(SNR_Cole(z_sim[i])*1.0/(1+z_sim[i])*dV_dz_dW[i]*delta_omega)#dn/dz is the SN per unit of redshift, comovil volume, solid anfle and fied of view


    n_Horiuchi[i]=dn_dz_Horiuchi[i]*((delta_t_redshift[i])*1.0/365.25) #In yr (SNR in yr)
    n_Cole[i]=dn_dz_Cole[i]*((delta_t_redshift[i])*1.0/365.25)
                               

plot(z_sim,n_Horiuchi,'b',z_sim,n_Cole,'c')
title('The number of detected SNe II per unit of redshift interval')
xlabel('redshift')
ylabel('num SN ')
legend(('Horiuchi','Cole'),2)
show()


# We integrate for all the redshift to have the number of SN
    
n_func_Horiuchi=interpolate.interp1d(z_sim,n_Horiuchi)
n_supernova_Horiuchi=integrate.quad(n_func_Horiuchi,min(z_sim),max(z_sim))[0]

n_func_Cole=interpolate.interp1d(z_sim,n_Cole)
n_supernova_Cole=integrate.quad(n_func_Cole,min(z_sim),max(z_sim))[0]


# number of SN versus redshift!!

bin_int=np.arange(0.01,max(z_sim)+0.01,0.01)
n_supernova_Horiuchi=[0]*(size(z_sim))
n_supernova_Cole=[0]*(size(z_sim))


for i in range(size(z_sim)-1):
    n_supernova_Horiuchi[i]=integrate.quad(n_func_Horiuchi,bin_int[i],bin_int[i+1])[0]
    n_supernova_Cole[i]=integrate.quad(n_func_Cole,bin_int[i],bin_int[i+1])[0]


ylabel(' SN/day (CDF)')
plot(z_sim,n_supernova_Horiuchi,'b',z_sim,n_supernova_Cole,'c')
legend(('Horiuchi','Cole'),2)
show()



Givea value of the maximum redshift used :1.0


NameError: name 'math' is not defined